In [1]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.common.action_chains import ActionChains
import winsound

In [ ]:
#URL parsing
#Check URL format:
#https://www.pets4homes.co.uk/sale/reptiles/local/
#https://www.pets4homes.co.uk/sale/reptiles/local/birmingham/
#https://www.pets4homes.co.uk/sale/reptiles/local/birmingham/page-11/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/local/page-35/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/durham/page-2/
#https://www.pets4homes.co.uk/sale/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/adoption/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/stud/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/sale/dogs/local/aberdeen/
#URL format is kind of pain to work with, probably should scrape by type of listing, type of pet and city separately.
#species or breed would be listed in the dataframe, no need to be scraped individually.
#so the format would be:
#https://www.pets4homes.co.uk/<type of listing>/<type of pets>/local/<city>/<page>
#page don't exist in URL for page 1.

In [ ]:
url_template = "https://www.pets4homes.co.uk/{}/{}/local/{}/"
listing_type = ['sale','adoption','stud']
pet_type = ['dogs','cats','reptiles','rodents','rabbits','horses','invertebrates','livestock', 'poultry', 'birds','fish']
cities = ['aberdeen','ayr_st-ives', 'barnsley_wimborne','basingstoke','bath','bedfordshire','birmingham','blackpool_plymouth',
         'bolton_appleby-in-westmorland', 'bournemouth', 'bradford_holsworthy', 'bridgend_brechin', 'brighton',
          'bristol_south-west-england', 'cambridge_gloucester', 'cardiff-county', 'carlisle', 'chelmsford', 'cheltenham',
         'chester', 'chesterfield', 'colchester', 'cornwall', 'coventry', 'crawley_witney', 'crewe', 'darlington', 'derby', 
         'devon', 'doncaster', 'dorset', 'dudley_cramlington', 'dundee', 'durham', 'eastbourne', 'edinburgh', 'essex', 'exeter',
         'glasgow', 'gloucester', 'grimsby', 'guildford', 'hampshire', 'hereford', 'hertfordshire', 'huddersfield']
browser = webdriver.Chrome(ChromeDriverManager().install())
page = 1

In [ ]:
for city in set (cities):
    for pets in set(pet_type):
        for listing in set(listing_type):
            url = url_template.format(listing, pets, city)
            options = webdriver.ChromeOptions()
            browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)
            browser.get(url)
            title = (
                WebDriverWait(driver=browser, timeout=10)
                .until(visibility_of_element_located((By.CSS_SELECTOR, "h1"))).text
            )
            content = browser.page_source
            soup = BeautifulSoup(content, 'lxml')
            print(url)
            sleep(random.randint(10,15))
            
            sleep(random.randint(10,30))

In [2]:
def check_captcha (soup):
    if 'I am human' in soup.get_text():
            print('captcha detected')
            duration = 5000  # milliseconds
            freq = 440  # Hz
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter
    else:
        print('no captcha detected, proceed to scrape')

In [ ]:
def check_if_empty (soup):
    if 'We found 0' in soup.get_text():
            print('no result found')
            

In [3]:
def scrape_header(soup):
    df = pd.DataFrame(columns=["Title","Description","photo_link","category", "price", "url",
                                              "seller_type", "seller_name", 'listing_type','pet_type'])
    for each in soup.find_all("script", type="application/ld+json"):
        entries = each.get_text()
        split = entries.split("\n")
        Title = split[3].replace('"name": "', '').replace('",', '')
        Description = split[4].replace('"description": "', '').replace('",', '').replace('\n,', '')
        Photo = split[5].replace('"image": "', '').replace('",', '')
        Category = split[7].replace('"category": "', '').replace('",', '')
        Price = split[11].replace('"price": "', '').replace('",', '')
        url =split[13].replace('"url": "', '').replace('",', '')
        seller_type = split[15].replace('"type": "', '').replace('",', '')
        seller_name = split[16].replace('"name":  "', '').replace('"', '')
        df = df.append({"Title":Title,"Description":Description,"photo_link":Photo,"category":Category, 
                                      "price":Price,"url":url, "seller_type":seller_type, "seller_name":seller_name,
                                      "listing_type":listing, "pet_type": pets},ignore_index=True)
    return df

In [4]:
def scrape_content(soup):
    df1 = pd.DataFrame(columns=["Title","price","species", "age", "gender", "description",
                                "seller_name", "seller_location", "seller_type", "listing_type","pet_type"])
    for each in soup.find_all("div", class_="mn aj"):
        entries = each.get_text()
        title = each.find('h2', {'class':'wn'}).text
        price = each.find('span', {'class':'xn'}).text
        info = each.find_all('span', {'class':'rv'})
        species = info[0].text
        try:
            age = info[1].text.replace('Age:  ', '')
        except:
            age = 'unknown'
        try:
            gender = info[2].text
        except:
            gender = 'unknown'
        description = each.find('span', {'class':'yn'}).text
        seller_name = each.find('span', {'class':'En'}).text
        seller_location = each.find('span', {'class':'Ct'}).text
        seller_type = each.find('div', {'class':'se sm te'}).text
        df1 = df1.append({"Title":title, "price":price, "species":species, "age":age, "gender":gender,
                              "description":description, "seller_name":seller_name, "seller_location": seller_location,
                              "seller_type":seller_type, "listing_type":listing, "pet_type": pets},ignore_index=True)
    return df1

# For testing page loops

In [5]:
url_template = "https://www.pets4homes.co.uk/{}/{}/local/{}/"
#https://www.pets4homes.co.uk/sale/dogs/local/huddersfield/page-2/
url_template_pages = "https://www.pets4homes.co.uk/{}/{}/local/{}/page-{}/"
listing_type = ['sale']
pet_type = ['dogs']
cities = ['aberdeen']

In [37]:
for city in set (cities):
    for pets in set(pet_type):
        for listing in set(listing_type):
            page = 1
            while page > 0:
                clear_output(wait=True)
                file = 'data_by_cities/header_'+str(city)+'.csv'
                file1 = 'data_by_cities/body_'+str(city)+'.csv'
                try: 
                    df_header = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
                    print('file successfully opened')
                except:
                    print('file not found, creating new file')
                    df_header = pd.DataFrame(columns=["Title","Description","photo_link","category", "price", "url",
                                              "seller_type", "seller_name", 'listing_type','pet_type'])
                    df_header.to_csv(file, sep='\t', encoding='utf-8')
                try: 
                    df_body = pd.read_csv(file1 , sep='\t', encoding='utf-8', index_col=0)
                    print('file successfully opened')
                except:
                    print('file not found, creating new file')
                    df_body = pd.DataFrame(columns=["Title","price","species", "age", "gender", "description",
                                                    "seller_name", "seller_location", "seller_type" , 
                                                    "listing_type","pet_type"])
                    df_body.to_csv(file1, sep='\t', encoding='utf-8')
                headersize = df_header.shape
                bodysize = df_body.shape
                if page == 1:
                    url = url_template.format(listing, pets, city)
                else:
                    url = url_template_pages.format(listing, pets, city, page)
                options = webdriver.ChromeOptions()
                browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)
                browser.get(url)
                title = (
                    WebDriverWait(driver=browser, timeout=15)
                    .until(visibility_of_element_located((By.CSS_SELECTOR, "h1"))).text
                )
                content = browser.page_source
                print(city,pets,listing,page)
                soup = BeautifulSoup(content, 'lxml')
                check_captcha(soup)
                if 'We found 0' in soup.get_text():
                    print('no result found')
                    break
                df_header = df_header.append(scrape_header(soup))
                df_body = df_body.append(scrape_content(soup))
                #drop duplicates
                df_header = df_header[~df_header.duplicated(subset=['Title','Description', 'photo_link','category','url','seller_type','listing_type','pet_type'])].reset_index(drop=True)
                df_header.to_csv(file, sep='\t', encoding='utf-8')
                df_body = df_body[~df_body.duplicated()].reset_index(drop=True)
                df_body.to_csv(file1, sep='\t', encoding='utf-8')
                if df_body.shape == bodysize:
                    if df_header.shape == headersize:
                        print('no new data')
                    else:
                        winsound.Beep(600, 1000)
                        input('html tags is likely changed, please kill program')
                next_page = soup.find('a', {'class':'Fb gh Jb'})
                if next_page == None:
                    print('next page not found')
                    break
                else:
                    print ('next page found')
                    print('going to next page')
                    sleep(random.randint(10,15))
                    page +=1
            print('going to next listing type')
            sleep(random.randint(10,15))
        print('going to next pet type')
        sleep(random.randint(10,15))
    print('going to next city')



====== WebDriver manager ======


file successfully opened
file successfully opened


Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


aberdeen dogs sale 5
no captcha detected, proceed to scrape
no new data
next page not found
going to next listing type
going to next pet type
going to next city


In [11]:
df_body.head(30)

,Title,price,species,age,gender,description,seller_name,seller_location,seller_type,listing_type,pet_type
0,unsexed green tree python + 60cm vivtek,"£1,000",Python Snake,3 years,Mixed,~3years old gtp biak x sorong\r\n\r\nTypical g...,Patryk M.,Aberdeen,Individual,sale,reptiles
1,Leopard gecko (f),£80,Gecko,1 year,Female,Selling this leopard gecko I've had the joy of...,Dana I.,Aberdeen,Individual,sale,reptiles
2,rexy,£100,Iguana,4 years,Male,rexy needs a new home ASP\r\n sadly I don't no...,Gillian W.,Aberdeen,Individual,sale,reptiles
3,Panther Chameleon and full set up,£550,Chameleon,9 months,Male,Blue is a 6 month old Nosey Mitsio chameleon. ...,Craig H.,Aberdeen,Individual,sale,reptiles
4,Hand reared white Indian ringnecks,£300,Parakeets,9 weeks,Mixed,I have 2 stunning white Indian ringnecks. 1 ma...,Dave T.,Aberdeen,Individual,sale,birds
5,beautiful Main Coone x kittens,£400,Maine Coon,6 weeks,2 male / 1 female,we have 3 fluffy bundles nearly ready for thei...,Andy B.,Aberdeen,Individual,sale,cats
6,Beautiful Sphynx kittens,£850,Sphynx,6 weeks,3 male / 2 female,"Sphynx kittens,\r\n\r\nMy beautiful Girl have ...",Natalie S.,Ellon,Individual,sale,cats
7,Bengal female for sale,£950,Bengal,3 months,1 female,Girl Waiting for Forever Home 🐆🏡\r\nThe girl ...,BENGAL PARADISE,Aberdeen,Breeder,sale,cats
8,Savannah F3,"£1,750",Savannah,15 weeks,unknown,Introducing Big F3 male Barra !\r\n \r\nFrom L...,Savannah,Ellon,Breeder,sale,cats
9,Savannah F3 Male,"£1,750",Savannah,15 weeks,2 male,Introducing F 3 male Jura From lady Starlet ...,Savannah,Ellon,Breeder,sale,cats


It works, let's make new notebook for tidier code

In [36]:
try:
    page1 = soup.find('a', {'class':'Fb gh Jb'})
    print(page1)
    print('next page found')
except:
    print('next page not found')

<a aria-label="Next page" class="Fb gh Jb" href="/sale/dogs/local/aberdeen/page-2/"><i class="xk icon-3-arrowrighy" style="--icon-size:12px"></i></a>
next page found


In [32]:
type(page1)

NoneType